# Easy

In [1]:
def func(input_list : list) -> dict:
    if len(input_list) == 0: # если список пустой
        return { 'mean' : 0, 'median' : 0, 'mode' : 0 }
    result = dict()
    frequency = dict()
    sum_of_list = 0
    for i in range(len(input_list)):
        sum_of_list += input_list[i]
        if input_list[i] in frequency.keys():
            frequency[input_list[i]] += 1
        else:
            frequency[input_list[i]] = 1
    result['mean'] = sum_of_list / len(input_list)
    input_list.sort()
    if len(input_list) % 2 == 0:
        result['median'] = (input_list[len(input_list) // 2] + input_list[len(input_list) // 2 - 1]) / 2
    else: 
        result['median'] = input_list[len(input_list) // 2]
    biggest_frequency = frequency[input_list[0]]
    result['mode'] = input_list[0]
    for i in frequency.keys():
        if frequency[i] > biggest_frequency:
            biggest_frequency = frequency[i]
            result['mode'] = i
    return result

In [2]:
func([0, 1, 1, 10, 5, 4, 3])

{'mean': 3.4285714285714284, 'median': 3, 'mode': 1}

In [3]:
func([0, 1, 1, 10, 5, 4])

{'mean': 3.5, 'median': 2.5, 'mode': 1}

In [4]:
func([])

{'mean': 0, 'median': 0, 'mode': 0}

# Medium

In [5]:
def parse_csv(file_content: str) -> list[dict]:
    result = list()
    with open('../data/dota_hero_stats.csv', 'r') as file:
        df = file.readlines()
    headings = df[0].split(',')
    headings[-1] = headings[-1][:-1] # обрезаем \n
    for i in range(1, len(df)):
        d = dict()
        current_line = df[i].split(',')
        for j in range(1, 7):
            if j == 2 or j == 3:
                d[headings[j]] = int(current_line[j])
            else: 
                d[headings[j]] = current_line[j]
        d[headings[-1]] = df[i][df[i].find('[')+2:df[i].find(']')-1].split("', '")
        result.append(d)
    return result

In [6]:
parsed_stats = parse_csv('../data/dota_hero_stats.csv')
print(parsed_stats[0])

{'attack_type': 'Melee', 'id': 1, 'legs': 2, 'localized_name': 'Anti-Mage', 'name': 'npc_dota_hero_antimage', 'primary_attr': 'agi', 'roles': ['Carry', 'Escape', 'Nuker']}


In [7]:
max_legs = parsed_stats[0]['legs']
id = 0
for i in range(len(parsed_stats)):
    if max_legs < parsed_stats[i]['legs']:
        max_legs = parsed_stats[i]['legs']
        id = i
print(parsed_stats[id])

{'attack_type': 'Melee', 'id': 61, 'legs': 8, 'localized_name': 'Broodmother', 'name': 'npc_dota_hero_broodmother', 'primary_attr': 'agi', 'roles': ['Carry', 'Pusher', 'Escape', 'Nuker']}


# Hard

Мера близости между персонажами - вещественное число от 0 до 1, где 0 значит, что персонажи идентичны, а 1 - не имеют ни одной общей черты. "Штраф" за непохожесть между двумя персонажами начисляется следущим образом:

1. **attack_type**: 0/1 - совпадает/не совпадает
2. **legs**: разница между количеством ног / 8, вещественное число от 0 до 1
3. **primary_attr**: 0/1 - совпадает/не совпадает
4. **roles**:  за каждую роль, которая есть у одного персонажа и нет у другого, начисляется 1 балл. Сумма баллов делится на 9, получаем вещественное число от 0 до 1

Результат = сумма баллов за все пункты / 4



In [8]:
import pandas as pd

/tmp/ipykernel_205882/4080736814.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [9]:
def parse_string(string : str) -> list:
    if len(string) == 0:
        return []
    string = string[1:-1]
    result = string.split(', ')
    return result

In [10]:
def diff_metric(hero_a : pd.DataFrame, hero_b : pd.DataFrame) -> float:
    result = 0
    result += int(not(hero_a['attack_type'] == hero_b['attack_type'])) + int(not(hero_a['primary_attr'] == hero_b['primary_attr']))
    result += abs(hero_a['legs'] - hero_b['legs']) / 8
    roles_a = parse_string(hero_a['roles'])
    roles_b = parse_string(hero_b['roles'])
    roles_diff = 0
    for role in roles_a:
        if role not in roles_b:
            roles_diff += 1
    for role in roles_b:
        if role not in roles_a:
            roles_diff += 1
    result += roles_diff / 9
    return result / 4

In [11]:
with open('../data/dota_hero_stats.csv', 'r') as file:
    df = pd.read_csv(file)
df.head()

,Unnamed: 0,attack_type,id,legs,localized_name,name,primary_attr,roles
0,0,Melee,1,2,Anti-Mage,npc_dota_hero_antimage,agi,"['Carry', 'Escape', 'Nuker']"
1,1,Melee,2,2,Axe,npc_dota_hero_axe,str,"['Initiator', 'Durable', 'Disabler', 'Jungler']"
2,2,Ranged,3,4,Bane,npc_dota_hero_bane,int,"['Support', 'Disabler', 'Nuker', 'Durable']"
3,3,Melee,4,2,Bloodseeker,npc_dota_hero_bloodseeker,agi,"['Carry', 'Disabler', 'Jungler', 'Nuker', 'Ini..."
4,4,Ranged,5,2,Crystal Maiden,npc_dota_hero_crystal_maiden,int,"['Support', 'Disabler', 'Nuker', 'Jungler']"


In [12]:
df_as_dict = df.to_dict(orient='index')

In [13]:
df_as_dict[0]

{'Unnamed: 0': 0,
 'attack_type': 'Melee',
 'id': 1,
 'legs': 2,
 'localized_name': 'Anti-Mage',
 'name': 'npc_dota_hero_antimage',
 'primary_attr': 'agi',
 'roles': "['Carry', 'Escape', 'Nuker']"}

In [14]:
df_as_dict[1]

{'Unnamed: 0': 1,
 'attack_type': 'Melee',
 'id': 2,
 'legs': 2,
 'localized_name': 'Axe',
 'name': 'npc_dota_hero_axe',
 'primary_attr': 'str',
 'roles': "['Initiator', 'Durable', 'Disabler', 'Jungler']"}

In [15]:
diff_metric(df_as_dict[0], df_as_dict[1])

0.4444444444444444

In [16]:
df_size = len(list(df_as_dict.keys()))-1
df_size

116

In [17]:
min_diff = 1
ans = []
for i in range(df_size):
    for j in (i + 1, df_size):
        cur_diff = diff_metric(df_as_dict[i], df_as_dict[j])
        if cur_diff < min_diff:
            min_diff = cur_diff
            ans = [i, j]
print(min_diff)
print(ans)

0.027777777777777776
[17, 18]


In [18]:
df_as_dict[17]

{'Unnamed: 0': 17,
 'attack_type': 'Melee',
 'id': 18,
 'legs': 2,
 'localized_name': 'Sven',
 'name': 'npc_dota_hero_sven',
 'primary_attr': 'str',
 'roles': "['Carry', 'Disabler', 'Initiator', 'Durable', 'Nuker']"}

In [19]:
df_as_dict[18]

{'Unnamed: 0': 18,
 'attack_type': 'Melee',
 'id': 19,
 'legs': 2,
 'localized_name': 'Tiny',
 'name': 'npc_dota_hero_tiny',
 'primary_attr': 'str',
 'roles': "['Carry', 'Nuker', 'Pusher', 'Initiator', 'Durable', 'Disabler']"}